# Goal Build all pinning services into more abstract API to upload and download dataset

* NFT Storage
* Pinata
* Eternum
* Estuary

## NFT Storage

In [13]:
import nftstorage

In [14]:
nft_storage = nftstorage.NFTStorage()

In [15]:
cred = nft_storage.get_creds()

In [16]:
nft_storage.upload_file("IMG_3990.JPG",cred)

({'ok': True,
  'value': {'cid': 'bafybeif36uqfqj5on23qfdl3nglc7ncgcbnz4ixpleomskg7pqn3xmlgfi',
   'created': '2022-03-09T04:03:21.867+00:00',
   'type': 'form-data',
   'scope': 'audiocentive',
   'files': [],
   'size': 2086898,
   'pin': {'cid': 'bafybeif36uqfqj5on23qfdl3nglc7ncgcbnz4ixpleomskg7pqn3xmlgfi',
    'created': '2022-03-09T04:03:21.867+00:00',
    'size': 2086898,
    'status': 'pinned'},
   'deals': [{'status': 'active',
     'lastChanged': '2022-03-09T11:01:51.318299+00:00',
     'chainDealID': 4380122,
     'datamodelSelector': 'Links/92/Hash/Links/193/Hash/Links/1/Hash',
     'statusText': 'containing sector active as of 2022-03-09 22:18:30 at epoch 1618597',
     'dealActivation': '2022-03-10T19:13:30+00:00',
     'dealExpiration': '2023-08-02T19:13:30+00:00',
     'miner': 'f01199442',
     'pieceCid': 'baga6ea4seaqlt3op73vmsqcservja27nsqjv32itwmwc46rjc5nfb7hp5agmeey',
     'batchRootCid': 'bafybeiepq37bwmlpinstbug2nr2ohob34wi5nt3mnmnjkqtxu3da6ruupi'},
    {'status'

In [334]:
nft_storage.get_file("bafkreigjqslso7qcejebq6vuyg5ezzkdja5euughikfga3hky2knwzmmhi",cred)

{'ok': True,
 'value': {'cid': 'bafkreigjqslso7qcejebq6vuyg5ezzkdja5euughikfga3hky2knwzmmhi',
  'created': '2022-03-09T02:06:48.313+00:00',
  'type': 'form-data',
  'scope': 'audiocentive',
  'files': [],
  'size': 9876,
  'pin': {'cid': 'bafkreigjqslso7qcejebq6vuyg5ezzkdja5euughikfga3hky2knwzmmhi',
   'created': '2022-03-09T02:06:48.313+00:00',
   'size': 9876,
   'status': 'pinned'},
  'deals': [{'status': 'active',
    'lastChanged': '2022-03-09T07:37:02.292312+00:00',
    'chainDealID': 4376541,
    'datamodelSelector': 'Links/108/Hash/Links/210/Hash/Links/1/Hash',
    'statusText': 'containing sector active as of 2022-03-09 12:27:30 at epoch 1617415',
    'dealActivation': '2022-03-10T13:09:30+00:00',
    'dealExpiration': '2023-08-02T13:09:30+00:00',
    'miner': 'f01611097',
    'pieceCid': 'baga6ea4seaqm5qvqfegcm7xhuedaehf77amqe4sji3qpz52iz4v3xwjsg6q6uki',
    'batchRootCid': 'bafybeifodhtjg5nj4ao7pfcowezsb7c5d3xpayq7bhhbg3qaxbsum4qs5q'},
   {'status': 'published',
    'lastCha

# IPFS

In [17]:
import ipfs

ipfs = ipfs.IPFS()

In [18]:
# w = ipfs.add("All_Gateway_Properties.csv")

# w.json()

In [19]:
f_name = ipfs.get_file("bafybeif36uqfqj5on23qfdl3nglc7ncgcbnz4ixpleomskg7pqn3xmlgfi",local_node=False)

Retrieved file hash bafybeif36uqfqj5on23qfdl3nglc7ncgcbnz4ixpleomskg7pqn3xmlgfi from https://dweb.link Response 200


In [20]:
response = f_name.content

In [105]:
read_file(response)

,No.,Address,City/State/Zip
0,1,7178 COLUMBIA GATEWAY DR,"COLUMBIA, MD 21046"
1,2,7220 LEE DEFOREST DR,"COLUMBIA, MD 21046"
2,3,7224 SOLAR WALK,"COLUMBIA, MD 21046"
3,4,7230 LEE DEFOREST DR,"COLUMBIA, MD 21046"
4,5,7061 COLUMBIA GATEWAY DR,"COLUMBIA, MD 21046"
...,...,...,...
208,209,7277 CALM SUNSET,"COLUMBIA, MD 21046"
209,210,9161 COMMERCE CENTER DR,"COLUMBIA, MD 21046"
210,211,8601 ROBERT FULTON DR,"COLUMBIA, MD 21046"
211,212,6741 COLUMBIA GATEWAY DR,"COLUMBIA, MD 21046"


# Pinata

In [94]:
import requests
import json

#Old API - https://docs.pinata.cloud/api-pinning/pinning-services-api
#New API - https://managed.mypinata.cloud/api/v1/api-docs/#/

#https://medium.com/pinata/how-to-pin-to-ipfs-effortlessly-ba3437b33885
class PinataV1:
    
    def get_creds(self):

        with open("creds.json") as f:

            cred = json.loads(f.read())

        return cred["Pinata"]

    def upload_file(self,fn,cred):

        base_url = "https://api.pinata.cloud/pinning/pinFileToIPFS"

        header = {
                  "pinata_api_key":cred["API Key"],
                    "pinata_secret_api_key":cred["API Secret"]
                  }
        
        
        #Filename,file, content_type, cookie expiration?
        files = {'file': (fn, open(fn, 'rb'),
                          "multipart/form-data", {'Expires': '0'})
                }


        response = requests.post(base_url, headers=header,files=files)

        return response.json(),response.status_code
    
    
    def pin(self,fn,cid,cred):

        base_url = "https://api.pinata.cloud/pinning/pinByHash"

        header = {
                  "pinata_api_key":cred["API Key"],
                    "pinata_secret_api_key":cred["API Secret"]
                  }
        
        data = {
                    # The "pinataMetadata" object will not be part of your content added to IPFS
                    # Pinata simply stores the metadata provided to help you easily query the content you've pinned with Pinata
                    "pinataMetadata": {
                        "name": fn,
                        "keyvalues": {}
                    },
                    "hashToPin": cid
            }
    


        response = requests.post(base_url, headers=header,json=data)

        return response.json(),response.status_code
    
    
    #Appears in documentation, but invalid endpoint
    def unpin(self,cid,cred):

        base_url = "https://api.pinata.cloud/pinning/unpin"

        header = {
                  "pinata_api_key":cred["API Key"],
                    "pinata_secret_api_key":cred["API Secret"]
                  }
        
        data = {"hashToUnpin":cid} #The hash of the content you wish to have Pinata unpin from its nodes.
    


        response = requests.delete(base_url, headers=header,data=data)

        return response.json(),response.status_code
    
    
    #Appears in documentation, but invalid endpoint
    def edit_hash(self,fn,cid,cred):

        base_url = "https://api.pinata.cloud/pinning/hashMetadata"

        header = {
                  "pinata_api_key":cred["API Key"],
                    "pinata_secret_api_key":cred["API Secret"]
                  }
        
        data = {
        "ipfsPinHash": cid,
        "name": fn,
        "keyvalues": {
            "newKey": 'newValue',
            "existingKey": 'newValue',
            "existingKeyToRemove": None
            }
        }
    


        response = requests.put(base_url, headers=header,data=data)

        return response.status_code
    
    def pin_policy(self,cid,cred,region="NYC1",replications=2):
        
        """
        FRA1 - Frankfurt, Germany (max 2 replications)
        NYC1 - New York City, USA (max 2 replications)
        """
        
        base_url = "https://api.pinata.cloud/pinning/hashPinPolicy"

        header = {
                  "pinata_api_key":cred["API Key"],
                    "pinata_secret_api_key":cred["API Secret"]
                  }
        
        data = {
                "ipfsPinHash": cid,
                "newPinPolicy": {
                    "regions": [
                        {
                            "id": region,
                            "desiredReplicationCount": replications
                        },
                    ]
                }
            }
        
        
        response = requests.put(base_url, headers=header,data=data)

        return response.json(),response.status_code
        
        
    def globalpin_policy(self,cred,region="NYC1",replications=2):
        
        """
        FRA1 - Frankfurt, Germany (max 2 replications)
        NYC1 - New York City, USA (max 2 replications)
        """
        
        base_url = "https://api.pinata.cloud/pinning/userPinPolicy"

        header = {
                  "pinata_api_key":cred["API Key"],
                    "pinata_secret_api_key":cred["API Secret"]
                  }
        
        data = {
                "newPinPolicy": {
                    "regions": [
                        {
                            "id": region,
                            "desiredReplicationCount": replications
                        },
                    ]
                }
            }
        
        
        response = requests.put(base_url, headers=header,data=data)

        return response.json(),response.status_code
    
    
    def get_pinned_jobs(self,cred,params):
        
        """
        "sort" - Sort the results by the date added to the pinning queue (see value options below)
        "ASC" - Sort by ascending dates
        "DESC" - Sort by descending dates
        "status" - Filter by the status of the job in the pinning queue (see potential statuses below)
        "prechecking" - Pinata is running preliminary validations on your pin request.
        "searching" - Pinata is actively searching for your content on the IPFS network. This may take some time if your content is isolated.
        "retrieving" - Pinata has located your content and is now in the process of retrieving it.
        "expired" - Pinata wasn't able to find your content after a day of searching the IPFS network. Please make sure your content is hosted on the IPFS network before trying to pin again.
        "over_free_limit" - Pinning this object would put you over the free tier limit. Please add a credit card to continue pinning content.
        "over_max_size" - This object is too large of an item to pin. If you're seeing this, please contact us for a more custom solution.
        "invalid_object" - The object you're attempting to pin isn't readable by IPFS nodes. Please contact us if you receive this, as we'd like to better understand what you're attempting to pin.
        "bad_host_node" - You provided a host node that was either invalid or unreachable. Please make sure all provided host nodes are online and reachable.
        "ipfs_pin_hash" - Retrieve the record for a specific IPFS hash
        "limit" - Limit the amount of results returned per page of results (default is 5, and max is 1000)
        "offset" - Provide the record offset for records being returned. This is how you retrieve records on additional pages (default is 0)
        """
        
        base_url = "https://api.pinata.cloud/pinning/pinJobs/"
        
        header = {
                  "pinata_api_key":cred["API Key"],
                    "pinata_secret_api_key":cred["API Secret"]
                  }
        
        
        response = requests.get(base_url, headers=header,params=params)

        return response.json(),response.status_code
        
    
    def get_pinned_files(self,cred,params):
        
        """
        Query Parameters = params
        
        hashContains: (string) - Filter on alphanumeric characters inside of pin hashes. Hashes which do not include the characters passed in will not be returned.

        pinStart: (must be in ISO_8601 format) - Exclude pin records that were pinned before the passed in "pinStart" datetime.

        pinEnd: (must be in ISO_8601 format) - Exclude pin records that were pinned after the passed in "pinEnd" datetime.

        unpinStart: (must be in ISO_8601 format) - Exclude pin records that were unpinned before the passed in "unpinStart" datetime.

        unpinEnd: (must be in ISO_8601 format) - Exclude pin records that were unpinned after the passed in "unpinEnd" datetime.

        pinSizeMin: (integer) - The minimum byte size that pin record you're looking for can have

        pinSizeMax: (integer) - The maximum byte size that pin record you're looking for can have

        status: (string) -
            * Pass in "all" for both pinned and unpinned records
            * Pass in "pinned" for just pinned records (hashes that are currently pinned)
            * Pass in "unpinned" for just unpinned records (previous hashes that are no longer being pinned on pinata)

        pageLimit: (integer) - This sets the amount of records that will be returned per API response. (Max 1000)

        pageOffset: (integer) - This tells the API how far to offset the record responses. For example, 
        if there's 30 records that match your query, and you passed in a pageLimit of 10, providing a pageOffset of 10 would return records 11-20.
        """
        
        base_url = "https://api.pinata.cloud/data/pinList?"
        
        header = {
                  "pinata_api_key":cred["API Key"],
                    "pinata_secret_api_key":cred["API Secret"]
                  }
        
        
        response = requests.get(base_url, headers=header,params=params)

        return response.json(),response.status_code
    
    
    def get_datausage(self,cred):
                
        header = {
                  "pinata_api_key":cred["API Key"],
                    "pinata_secret_api_key":cred["API Secret"]
                  }
        
        base_url = "https://api.pinata.cloud/data/userPinnedDataTotal"
        
        
        response = requests.get(base_url, headers=header,params=params)

        return response.json(),response.status_code

        
        

In [96]:
pinata = PinataV1()

In [97]:
cred = pinata.get_creds()

In [84]:
pinata.upload_file("storage/ipfs.py",cred)

({'IpfsHash': 'QmRxu3HoofyTntXhCSXBv98tPDWLdgNee32Z1nMQcCbxU7',
  'PinSize': 1640,
  'Timestamp': '2022-03-11T20:21:21.515Z'},
 200)

In [87]:
pinata.pin("test1.py","QmRxu3HoofyTntXhCSXBv98tPDWLdgNee32Z1nMQcCbxU7",cred)

({'id': 'dca644fe-7a6f-4d21-b203-8dba54e7060f',
  'ipfsHash': 'QmRxu3HoofyTntXhCSXBv98tPDWLdgNee32Z1nMQcCbxU7',
  'status': 'prechecking',
  'name': 'test1.py'},
 200)

In [76]:
pinata.unpin("bafybeif36uqfqj5on23qfdl3nglc7ncgcbnz4ixpleomskg7pqn3xmlgfi",cred)

({'error': {'reason': 'INVALID_ROUTE',
   'details': 'The provided route does not match a valid Pinata endpoint'}},
 404)

In [78]:
pinata.edit_hash("test.py","bafybeif36uqfqj5on23qfdl3nglc7ncgcbnz4ixpleomskg7pqn3xmlgfi",cred)

200

In [79]:
f_name = ipfs.get_file("QmZ8bNMNXodiWRJDvDHNCAE1EsnbaiTZaQJafm2pSTYBFh",local_node=False)

read_file(f_name.content)

NameError: name 'ipfs' is not defined

In [80]:
pinata.pin_policy("bafybeif36uqfqj5on23qfdl3nglc7ncgcbnz4ixpleomskg7pqn3xmlgfi",cred,region="FRA1",replications=1)

({'error': 'Pin policy is invalid. Please provide a valid pin policy',
  'message': 'Corrupted pin policy'},
 400)

In [81]:
pinata.globalpin_policy(cred,region="FRA1",replications=1)

({'error': {'reason': 'NOT_PERMITTED',
   'details': 'API key does not have required permissions for this endpoint'}},
 403)

## Request Files

In [85]:
import pandas as pd
params = {"status":"pinned","hashContains":None,"pinStart":None,
          "pinEnd":None, "unpinStart":None,"unpinEnd":None,
         "pinSizeMin":None,"pinSizeMax":None,"pageLimit":None,
          "pageOffset":None,
         }

pf,status = pinata.get_pinned_files(cred,params)
pd.DataFrame(pf["rows"])

,id,ipfs_pin_hash,size,user_id,date_pinned,date_unpinned,metadata,regions
0,a4f39d27-00b9-4092-83e6-3a4d9d9e202d,QmRxu3HoofyTntXhCSXBv98tPDWLdgNee32Z1nMQcCbxU7,1640,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-03-11T20:21:21.515Z,None,"{'name': 'storage', 'keyvalues': None}","[{'regionId': 'NYC1', 'currentReplicationCount..."
1,926f2e41-2420-4535-82c9-14e9f244a435,QmRE9W9efQjWgb9rABf4gU2SG48aabfvApUYJJBFE1BSzP,2350,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-03-11T20:20:00.213Z,None,"{'name': 'storage', 'keyvalues': None}","[{'regionId': 'NYC1', 'currentReplicationCount..."
2,721300ed-fd44-43ef-87a4-9ed2c3c3e2b9,bafybeif36uqfqj5on23qfdl3nglc7ncgcbnz4ixpleoms...,2086898,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-03-11T02:14:04.863Z,None,"{'name': 'test.py', 'keyvalues': {'0': 'newKey...","[{'regionId': 'NYC1', 'currentReplicationCount..."
3,3a79ae10-7392-4f0f-bafc-70543bdd6ada,QmWvvVY9meNZ6b6UXr8Vy7J2Z4eSS4YpPJ5nyrgfd7ZFMP,2290,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-03-10T16:49:18.236Z,None,"{'name': 'happy.py', 'keyvalues': {'0': 'newKe...","[{'regionId': 'NYC1', 'currentReplicationCount..."
4,147797ca-a95a-4be0-8acd-5242d31cfe8f,QmZ8bNMNXodiWRJDvDHNCAE1EsnbaiTZaQJafm2pSTYBFh,9887,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-03-10T15:57:40.831Z,None,"{'name': 'All_Gateway_Properties.csv', 'keyval...","[{'regionId': 'NYC1', 'currentReplicationCount..."


## Requesting Pin Jobs

In [89]:
import pandas as pd

"""
"sort" - Sort the results by the date added to the pinning queue (see value options below)
"ASC" - Sort by ascending dates
"DESC" - Sort by descending dates
"status" - Filter by the status of the job in the pinning queue (see potential statuses below)
"prechecking" - Pinata is running preliminary validations on your pin request.
"searching" - Pinata is actively searching for your content on the IPFS network. This may take some time if your content is isolated.
"retrieving" - Pinata has located your content and is now in the process of retrieving it.
"expired" - Pinata wasn't able to find your content after a day of searching the IPFS network. Please make sure your content is hosted on the IPFS network before trying to pin again.
"over_free_limit" - Pinning this object would put you over the free tier limit. Please add a credit card to continue pinning content.
"over_max_size" - This object is too large of an item to pin. If you're seeing this, please contact us for a more custom solution.
"invalid_object" - The object you're attempting to pin isn't readable by IPFS nodes. Please contact us if you receive this, as we'd like to better understand what you're attempting to pin.
"bad_host_node" - You provided a host node that was either invalid or unreachable. Please make sure all provided host nodes are online and reachable.
"ipfs_pin_hash" - Retrieve the record for a specific IPFS hash
"limit" - Limit the amount of results returned per page of results (default is 5, and max is 1000)
"offset" - Provide the record offset for records being returned. This is how you retrieve records on additional pages (default is 0)
"""
params = {"sort":"DESC","status":None,"prechecking":None,
          "searching":None,"retrieving":None,
          "expired":None,"over_free_limit":None,
          "over_max_size":None,"invalid_object":None,
          "bad_host_node":None,"ipfs_pin_hash":None,
          "limit":None,"offset":None
         }

pf,status = pinata.get_pinned_jobs(cred,params)
pf

{'count': 0, 'rows': []}

In [98]:
pinata.get_datausage(cred)

({'pin_count': 5,
  'pin_size_total': '2103065',
  'pin_size_with_replications_total': '2103065'},
 200)

In [104]:
#convert bytes to MB

2103065 * 10**-6

2.103065